In [3]:
# Colab-ready kmeans_kanpur.py (notebook-friendly argparse fix)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
)
import argparse
import sys
import os

# --- helper funcs (same as before) ---
def load_data(path):
    df = pd.read_csv(path)
    if 'datetime' not in df.columns or 'PM2.5' not in df.columns:
        raise ValueError("CSV must contain columns 'datetime' and 'PM2.5'")
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    df = df.dropna(subset=['datetime', 'PM2.5']).reset_index(drop=True)
    df['PM2.5'] = pd.to_numeric(df['PM2.5'], errors='coerce')
    df = df.dropna(subset=['PM2.5']).reset_index(drop=True)
    return df

def add_time_features(df):
    df['hour'] = df['datetime'].dt.hour
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow'] = df['datetime'].dt.dayofweek
    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)
    return df

def run_kmeans(X, k, random_state=42):
    km = KMeans(n_clusters=k, random_state=random_state, n_init=10)
    labels = km.fit_predict(X)
    centers = km.cluster_centers_
    inertia = km.inertia_
    return km, labels, centers, inertia

def compute_metrics(df, X_original, labels, centers):
    if 'PM2.5' in X_original.columns:
        pm25_true = X_original['PM2.5'].values
        # compute centroid PM2.5 by cluster mean (robust)
        pm25_centroid_vals = np.zeros_like(pm25_true, dtype=float)
        for lab in np.unique(labels):
            mask = labels == lab
            pm25_centroid_vals[mask] = pm25_true[mask].mean()
        rmse = np.sqrt(mean_squared_error(pm25_true, pm25_centroid_vals))
        mae = mean_absolute_error(pm25_true, pm25_centroid_vals)
        nonzero = pm25_true != 0
        mape = (np.abs((pm25_true[nonzero] - pm25_centroid_vals[nonzero]) / pm25_true[nonzero])).mean() * 100.0 if nonzero.sum()>0 else np.nan
    else:
        rmse = mae = mape = np.nan

    X_for_scores = X_original.drop(columns=['datetime']) if 'datetime' in X_original.columns else X_original
    try:
        sil = silhouette_score(X_for_scores, labels)
    except Exception:
        sil = np.nan
    try:
        ch = calinski_harabasz_score(X_for_scores, labels)
    except Exception:
        ch = np.nan
    try:
        db = davies_bouldin_score(X_for_scores, labels)
    except Exception:
        db = np.nan

    return {
        'RMSE': float(rmse),
        'MAE': float(mae),
        'AMAT (reported as MAE)': float(mae),
        'MAPE (%)': float(mape) if not np.isnan(mape) else None,
        'Silhouette': float(sil) if not np.isnan(sil) else None,
        'Calinski-Harabasz': float(ch) if not np.isnan(ch) else None,
        'Davies-Bouldin': float(db) if not np.isnan(db) else None
    }

def plot_results(df, labels, centers_pm25=None, out_prefix="kmeans_output"):
    df_plot = df.copy()
    df_plot['cluster'] = labels
    plt.figure(figsize=(12,5))
    for c in sorted(df_plot['cluster'].unique()):
        sub = df_plot[df_plot['cluster']==c]
        plt.scatter(sub['datetime'], sub['PM2.5'], s=10, label=f'cluster {c}', alpha=0.7)
    plt.xlabel('datetime')
    plt.ylabel('PM2.5')
    plt.title('PM2.5 time series colored by KMeans cluster')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{out_prefix}_timeseries_by_cluster.png", dpi=150)
    print(f"Saved plot: {out_prefix}_timeseries_by_cluster.png")
    plt.close()

    plt.figure(figsize=(8,5))
    df_plot.boxplot(column='PM2.5', by='cluster')
    plt.title('PM2.5 distribution by cluster')
    plt.suptitle('')
    plt.xlabel('cluster')
    plt.ylabel('PM2.5')
    plt.tight_layout()
    plt.savefig(f"{out_prefix}_boxplot_pm25_by_cluster.png", dpi=150)
    print(f"Saved plot: {out_prefix}_boxplot_pm25_by_cluster.png")
    plt.close()

    if centers_pm25 is not None:
        plt.figure(figsize=(6,4))
        plt.bar(range(len(centers_pm25)), centers_pm25)
        plt.xlabel('cluster')
        plt.ylabel('centroid PM2.5')
        plt.title('Cluster centroids (PM2.5)')
        plt.tight_layout()
        plt.savefig(f"{out_prefix}_centroids_pm25.png", dpi=150)
        print(f"Saved plot: {out_prefix}_centroids_pm25.png")
        plt.close()

def main(args):
    df = load_data(args.input)
    df = add_time_features(df)

    if args.feature_mode == 'pm25':
        features = df[['PM2.5']].copy()
    elif args.feature_mode == 'pm25_hour':
        features = df[['PM2.5','hour_sin','hour_cos']].copy()
    elif args.feature_mode == 'pm25_hour_dow':
        features = df[['PM2.5','hour_sin','hour_cos','dow_sin','dow_cos']].copy()
    else:
        raise ValueError("Unknown feature_mode")

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    km, labels, centers_scaled, inertia = run_kmeans(X, args.k, random_state=args.random_state)
    centers = scaler.inverse_transform(centers_scaled)

    try:
        pm25_centers = centers[:, 0]
    except Exception:
        pm25_centers = None

    df_metrics = df[['datetime','PM2.5']].copy()
    df_metrics['label'] = labels

    metrics = compute_metrics(df_metrics, df_metrics[['datetime','PM2.5']], labels, centers)

    print("\nKMeans results:")
    print(f" - input file: {args.input}")
    print(f" - n_samples: {len(df)}")
    print(f" - k clusters: {args.k}")
    print(f" - inertia (WCSS): {inertia:.4f}")
    print("\nMetrics:")
    for k,v in metrics.items():
        print(f"  {k}: {v}")

    unique, counts = np.unique(labels, return_counts=True)
    print("\nCluster counts:")
    for u,cnt in zip(unique, counts):
        center_val = pm25_centers[u] if pm25_centers is not None else np.nan
        print(f" - cluster {u}: count={cnt}, centroid PM2.5 (approx)={center_val:.3f}")

    plot_results(df, labels, centers_pm25=pm25_centers, out_prefix=args.output_prefix)

    out_csv = f"{args.output_prefix}_labeled.csv"
    df_out = df.copy()
    df_out['cluster'] = labels
    df_out.to_csv(out_csv, index=False)
    print(f"Saved labeled data: {out_csv}")

# ----------------- argparse (notebook-safe) -----------------
parser = argparse.ArgumentParser(description='KMeans clustering for kanpur.csv')
parser.add_argument('--input', '-i', default='kanpur.csv', help='Input CSV path (default: kanpur.csv)')
parser.add_argument('--k', '-k', type=int, default=3, help='Number of clusters (default: 3)')
parser.add_argument('--feature-mode', '-f', choices=['pm25','pm25_hour','pm25_hour_dow'], default='pm25',
                    help='Feature set: "pm25" only, "pm25_hour" adds hour cyclical features, "pm25_hour_dow" adds day-of-week.')
parser.add_argument('--output-prefix', '-o', default='kmeans_kanpur', help='Output prefix for plots/csv')
parser.add_argument('--random-state', type=int, default=42, help='Random seed for reproducibility')

# Notebook / Colab detection and safe parsing:
is_notebook = ('ipykernel' in sys.modules) or ('google.colab' in sys.modules)
if is_notebook:
    # Option A: use defaults (no CLI parsing from sys.argv) -> uncomment the line below to use defaults
    args = parser.parse_args([])

    # Option B: OR provide a custom list of CLI args (example)
    # custom_cli = ['--input','/content/kanpur.csv','--k','4','--feature-mode','pm25_hour','--output-prefix','out_kmeans']
    # args = parser.parse_args(custom_cli)
else:
    args = parser.parse_args()

# run
if __name__ == "__main__":
    # ensure file exists in Colab environment
    if not os.path.exists(args.input):
        print(f"Warning: input file '{args.input}' not found in current folder. Please upload kanpur.csv to the working directory.")
    main(args)



KMeans results:
 - input file: kanpur.csv
 - n_samples: 109
 - k clusters: 3
 - inertia (WCSS): 12.9142

Metrics:
  RMSE: 4.2264451067405355
  MAE: 3.518646439207691
  AMAT (reported as MAE): 3.518646439207691
  MAPE (%): 7.067621874438276
  Silhouette: 0.6212358366682833
  Calinski-Harabasz: 394.3383410513308
  Davies-Bouldin: 0.4771596187787739

Cluster counts:
 - cluster 0: count=48, centroid PM2.5 (approx)=52.900
 - cluster 1: count=27, centroid PM2.5 (approx)=71.185
 - cluster 2: count=34, centroid PM2.5 (approx)=40.188
Saved plot: kmeans_kanpur_timeseries_by_cluster.png
Saved plot: kmeans_kanpur_boxplot_pm25_by_cluster.png
Saved plot: kmeans_kanpur_centroids_pm25.png
Saved labeled data: kmeans_kanpur_labeled.csv


<Figure size 800x500 with 0 Axes>